In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
from pathlib import Path
import re

files = []

for path in Path('imagens').rglob('*.*'):
    for i in re.findall(r"([A-Z]{2,4}-[0-9]{2})", path.name):
        files.append([i, path.as_posix()])
#     print(re.findall(r"([A-Z]{2,4}-[0-9]{2})", path.name), path )

In [3]:
df_imagens = pd.DataFrame(files, columns=['cod', 'path'])

In [4]:
df_cods = pd.DataFrame(df_imagens.groupby('cod')['path'].apply(list))

In [5]:
df_imagens = pd.DataFrame(list(map(lambda x: x, df_cods.path))).add_prefix('imagem_')

In [6]:
df_imagens['cod'] = df_cods.index

In [7]:
df_imagens.set_index('cod')

,imagem_0,imagem_1,imagem_2,imagem_3,imagem_4,imagem_5,imagem_6,imagem_7
cod,,,,,,,,
AUR-00,imagens/BOTUJURU/EQUIPAMENTOS NOVOS/AUR-00 Pra...,imagens/BOTUJURU/EQUIPAMENTOS NOVOS/AUR-00 Pra...,None,None,None,None,None,None
AUR-01,imagens/JORDANOPOLIS/AUR-01 Praça Domingos Van...,imagens/JORDANOPOLIS/AUR-01 Praça Domingos Van...,imagens/JORDANOPOLIS/AUR-01 Praça Domingos Van...,imagens/JORDANOPOLIS/AUR-01 Praça Domingos Van...,None,None,None,None
AUR-02,imagens/JORDANOPOLIS/AUR-02 Praça Ronald. De C...,imagens/JORDANOPOLIS/AUR-02 Praça Ronald. De C...,imagens/JORDANOPOLIS/AUR-02 Praça Ronald. De C...,imagens/JORDANOPOLIS/AUR-02 Praça Ronald. De C...,None,None,None,None
AUR-03,imagens/MONTANHÃO/AUR-03 Praça Henrique Trinda...,imagens/MONTANHÃO/AUR-03 Praça Henrique Trinda...,imagens/MONTANHÃO/AUR-03 Praça Henrique Trinda...,None,None,None,None,None
AUR-04,imagens/MONTANHÃO/AUR-04 Praça Vicente de Paul...,imagens/MONTANHÃO/AUR-04 Praça Vicente de Paul...,imagens/MONTANHÃO/AUR-04 Praça Vicente de Paul...,imagens/MONTANHÃO/AUR-04 Praça Vicente de Paul...,imagens/MONTANHÃO/AUR-04 Praça Vicente de Paul...,imagens/MONTANHÃO/AUR-04 Praça Vicente de Paul...,None,None
...,...,...,...,...,...,...,...,...
SAU-15,imagens/INDEPENDÊNCIA/EQUIPAMENTOS EXISTENTES/...,imagens/INDEPENDÊNCIA/EQUIPAMENTOS EXISTENTES/...,imagens/INDEPENDÊNCIA/EQUIPAMENTOS EXISTENTES/...,imagens/INDEPENDÊNCIA/EQUIPAMENTOS EXISTENTES/...,imagens/INDEPENDÊNCIA/EQUIPAMENTOS EXISTENTES/...,imagens/INDEPENDÊNCIA/EQUIPAMENTOS EXISTENTES/...,None,None
SAU-16,imagens/PAULICÉIA/EQUIPAMENTOS EXISTENTES/SAU-...,imagens/PAULICÉIA/EQUIPAMENTOS EXISTENTES/SAU-...,imagens/PAULICÉIA/EQUIPAMENTOS EXISTENTES/SAU-...,imagens/PAULICÉIA/EQUIPAMENTOS EXISTENTES/SAU-...,imagens/PAULICÉIA/EQUIPAMENTOS EXISTENTES/SAU-...,imagens/PAULICÉIA/EQUIPAMENTOS EXISTENTES/SAU-...,None,None
SAU-17,imagens/PAULICÉIA/EQUIPAMENTOS EXISTENTES/SAU-...,imagens/PAULICÉIA/EQUIPAMENTOS EXISTENTES/SAU-...,imagens/PAULICÉIA/EQUIPAMENTOS EXISTENTES/SAU-...,imagens/PAULICÉIA/EQUIPAMENTOS EXISTENTES/SAU-...,imagens/PAULICÉIA/EQUIPAMENTOS EXISTENTES/SAU-...,imagens/PAULICÉIA/EQUIPAMENTOS EXISTENTES/SAU-...,None,None


In [8]:
df_equip = pd.read_csv('./GIS/planilhas/equipamentos-XY.csv', delimiter=';', decimal=',')

In [9]:
df_equip

,BAIRRO,NOME,X,Y,TIPO,ENDEREÇO,COD DO LOCAL,PRIORIDADE,ESTADO DO EQUIPAMENTO,Video_0
0,BATISTINI,EMEB 1 OCTAVIO EDGARD DE OLIVEIRA,-46.610875,-23.771093,EDUCAÇÃO,R. João Saldanha 424,ED-31,NaN,EXISTENTE,videos/batistini/equipamentos/EMEB 1 OCTAVIO E...
1,BATISTINI,EMEB ANTONIO DOS SANTOS FARIAS,-46.599049,-23.775686,EDUCAÇÃO,"R. Irati, 11",ED-32,NaN,EXISTENTE,videos/batistini/equipamentos/EMEB 2 ANTONIO D...
2,BATISTINI,EMEB CLEIA MARIA TEURES DE SOUZA,-46.599302,-23.775548,EDUCAÇÃO,"R. Irati, 2",ED-33,NaN,EXISTENTE,videos/batistini/equipamentos/EMEB 3 CLEIA MAR...
3,BATISTINI,EMEB MANOEL DE BARROS,-46.596923,-23.776791,EDUCAÇÃO,"Rua Canada, 1",ED-34,NaN,EXISTENTE,videos/batistini/equipamentos/EMEB 4 MANOEL DE...
4,BATISTINI,EMEB PROFESSOR WALDEMAR CANCIANI,-46.590605,-23.769596,EDUCAÇÃO,"R. José Martins Fernandes, 86",ED-35,NaN,EXISTENTE,videos/batistini/equipamentos/EMEB 5 PROFESSOR...
...,...,...,...,...,...,...,...,...,...,...
97,TABOÃO,Taboão - E. Infantil/ Fundamental/ ESCOLA EST...,-46.605547,-23.660592,EDUCAÇÃO,"Av. do Taboão, 3550 - Vila Santa Luzia, CEP 09...",NaN,NaN,REFORMA,videos/taboao/equipamentos/ESCOLA ESTADUAL PRO...
98,TABOÃO,Taboão - Creche/Integral,-46.600345,-23.656835,EDUCAÇÃO,"R. dos Alpes, alt. 109 - Suiço, CEP 09663-070",NaN,NaN,NOVO,videos/taboao/equipamentos/Taboão - CrecheInte...
99,MONTANHÃO,Ecoponto,-46.521962,-23.714673,COLETA SELETIVA,NaN,NaN,NaN,EXISTENTE,NaN
100,DOS ALVARENGA,Ecoponto,-46.598939,-23.741218,COLETA SELETIVA,NaN,NaN,NaN,EXISTENTE,NaN


In [10]:
df_equip['cod'] = df_equip['COD DO LOCAL ']
# df_equip.columns

In [11]:
df_equip = pd.merge(df_equip, df_imagens.set_index('cod'), on='cod', how='left')

In [12]:
gdf_equip = gpd.GeoDataFrame(df_equip, geometry=gpd.points_from_xy(df_equip.X, df_equip.Y))

In [13]:
gdf_equip.drop(['X', 'Y', 'COD DO LOCAL '], axis=1).to_file('GIS/equipamentos-com-fotos.gpkg', driver='GPKG')

In [14]:
for b in gdf_equip.BAIRRO.unique():
    b_formatado = b.lower().replace(" ", "_")
    gdf_equip.drop(['X', 'Y', 'COD DO LOCAL '], axis=1).loc[gdf_equip.BAIRRO == b].to_file(f'GIS/equipamentos-com-fotos-{b_formatado}.gpkg', driver='GPKG')
    gdf_equip.drop(['X', 'Y', 'COD DO LOCAL '], axis=1).loc[gdf_equip.BAIRRO == b].to_file(f'GIS/equipamentos-com-fotos-{b_formatado}.geojson', driver='GeoJSON')

In [15]:
gdf_equip

,BAIRRO,NOME,X,Y,TIPO,ENDEREÇO,COD DO LOCAL,PRIORIDADE,ESTADO DO EQUIPAMENTO,Video_0,cod,imagem_0,imagem_1,imagem_2,imagem_3,imagem_4,imagem_5,imagem_6,imagem_7,geometry
0,BATISTINI,EMEB 1 OCTAVIO EDGARD DE OLIVEIRA,-46.610875,-23.771093,EDUCAÇÃO,R. João Saldanha 424,ED-31,NaN,EXISTENTE,videos/batistini/equipamentos/EMEB 1 OCTAVIO E...,ED-31,imagens/BATISTINI/EQUIPAMENTOS EXISTENTES/ED-3...,imagens/BATISTINI/EQUIPAMENTOS EXISTENTES/ED-3...,imagens/BATISTINI/EQUIPAMENTOS EXISTENTES/ED-3...,imagens/BATISTINI/EQUIPAMENTOS EXISTENTES/ED-3...,imagens/BATISTINI/EQUIPAMENTOS EXISTENTES/ED-3...,imagens/BATISTINI/EQUIPAMENTOS EXISTENTES/ED-3...,None,None,POINT (-46.61087 -23.77109)
1,BATISTINI,EMEB ANTONIO DOS SANTOS FARIAS,-46.599049,-23.775686,EDUCAÇÃO,"R. Irati, 11",ED-32,NaN,EXISTENTE,videos/batistini/equipamentos/EMEB 2 ANTONIO D...,ED-32,imagens/BATISTINI/EQUIPAMENTOS EXISTENTES/ED-3...,imagens/BATISTINI/EQUIPAMENTOS EXISTENTES/ED-3...,imagens/BATISTINI/EQUIPAMENTOS EXISTENTES/ED-3...,imagens/BATISTINI/EQUIPAMENTOS EXISTENTES/ED-3...,imagens/BATISTINI/EQUIPAMENTOS EXISTENTES/ED-3...,imagens/BATISTINI/EQUIPAMENTOS EXISTENTES/ED-3...,None,None,POINT (-46.59905 -23.77569)
2,BATISTINI,EMEB CLEIA MARIA TEURES DE SOUZA,-46.599302,-23.775548,EDUCAÇÃO,"R. Irati, 2",ED-33,NaN,EXISTENTE,videos/batistini/equipamentos/EMEB 3 CLEIA MAR...,ED-33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-46.59930 -23.77555)
3,BATISTINI,EMEB MANOEL DE BARROS,-46.596923,-23.776791,EDUCAÇÃO,"Rua Canada, 1",ED-34,NaN,EXISTENTE,videos/batistini/equipamentos/EMEB 4 MANOEL DE...,ED-34,imagens/BATISTINI/EQUIPAMENTOS EXISTENTES/ED-3...,imagens/BATISTINI/EQUIPAMENTOS EXISTENTES/ED-3...,imagens/BATISTINI/EQUIPAMENTOS EXISTENTES/ED-3...,imagens/BATISTINI/EQUIPAMENTOS EXISTENTES/ED-3...,None,None,None,None,POINT (-46.59692 -23.77679)
4,BATISTINI,EMEB PROFESSOR WALDEMAR CANCIANI,-46.590605,-23.769596,EDUCAÇÃO,"R. José Martins Fernandes, 86",ED-35,NaN,EXISTENTE,videos/batistini/equipamentos/EMEB 5 PROFESSOR...,ED-35,imagens/BATISTINI/EQUIPAMENTOS EXISTENTES/ED-3...,imagens/BATISTINI/EQUIPAMENTOS EXISTENTES/ED-3...,imagens/BATISTINI/EQUIPAMENTOS EXISTENTES/ED-3...,imagens/BATISTINI/EQUIPAMENTOS EXISTENTES/ED-3...,imagens/BATISTINI/EQUIPAMENTOS EXISTENTES/ED-3...,imagens/BATISTINI/EQUIPAMENTOS EXISTENTES/ED-3...,None,None,POINT (-46.59060 -23.76960)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,TABOÃO,Taboão - E. Infantil/ Fundamental/ ESCOLA EST...,-46.605547,-23.660592,EDUCAÇÃO,"Av. do Taboão, 3550 - Vila Santa Luzia, CEP 09...",NaN,NaN,REFORMA,videos/taboao/equipamentos/ESCOLA ESTADUAL PRO...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-46.60555 -23.66059)
98,TABOÃO,Taboão - Creche/Integral,-46.600345,-23.656835,EDUCAÇÃO,"R. dos Alpes, alt. 109 - Suiço, CEP 09663-070",NaN,NaN,NOVO,videos/taboao/equipamentos/Taboão - CrecheInte...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-46.60034 -23.65683)
99,MONTANHÃO,Ecoponto,-46.521962,-23.714673,COLETA SELETIVA,NaN,NaN,NaN,EXISTENTE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-46.52196 -23.71467)
100,DOS ALVARENGA,Ecoponto,-46.598939,-23.741218,COLETA SELETIVA,NaN,NaN,NaN,EXISTENTE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-46.59894 -23.74122)
